In [29]:
# ライブラリのインポート
import wandb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

import sys
from pathlib import Path

# プロジェクトルートを sys.path に追加
project_root = Path.cwd()
sys.path.insert(0, str(project_root / "src"))
# プロジェクトルートの設定をしておかないと my_utils が import できない
from my_utils.array_logger import ArrayReader

print("✓ ライブラリインポート完了")

api = wandb.Api()

def load_array_log_df(run_id: str, array_name: str) -> pd.DataFrame:
    # ArrayReader でデータを DataFrame として読み込む
    db_path = project_root / "results" / "array_logs" / run_id / f"{array_name}.db"
    reader = ArrayReader(str(db_path))
    reader.open()
    df = reader.to_dataframe()
    reader.close()
    # print(f"✓ 読み込み完了！")
    # print(f"\nデータ形状: {df.shape}")
    # print(f"\narrayの形状: {df['array'].iloc[0].shape}")
    # print(f"\n最初の3行:")
    # print(df.head(10))
    return df

✓ ライブラリインポート完了


In [35]:
import pprint

filters = {
    "$and": [
        {"created_at": {"$gte": "2026-01-04T00:00:00"}}, # gte: Greater Than or Equal (以上)
        {"state": "finished"},
        {"display_name": "10a10f_lc1"},
        # {"config.enable_fixed_wait_action_prob": True},
        # {"config.fixed_wait_action_prob": 0},
    ]
}
runs = api.runs("kazuyasakakibara/FoodBank2025", filters=filters)

print("filters:")
pprint.pprint(filters)
print(f"{len(runs)} 件の実験データを取得しました。")

# # test_completedの値でrunを分類
# completed_values = [run.summary.get("test_completed") for run in runs]
# completed_counts = pd.Series(completed_values).value_counts().sort_index()
# print(completed_counts)

# print("\n=== test_completed別の割合 ===")
# completed_ratio = completed_counts / len(runs)
# for completed, ratio in completed_ratio.items():
#     print(f"{completed}: {ratio:.2f}")
# print(f"（成功率: {completed_ratio[1]:.2f}）")

filters:
{'$and': [{'created_at': {'$gte': '2026-01-04T00:00:00'}},
          {'state': 'finished'},
          {'display_name': '10a10f_lc1'}]}
30 件の実験データを取得しました。


## ランダム行動時の選択行動カウント

In [ ]:
for i, run in enumerate(runs):
    run_id = run.id
    print(f"\n=== run_id: {run_id}, name: {run.name} ({i+1}/{len(runs)}) ===")

    avail_actions_df = load_array_log_df(run_id, "avail_actions")
    pick_random_df = load_array_log_df(run_id, "pick_random")
    picked_actions_df = load_array_log_df(run_id, "picked_actions")

    # 全行の配列を一度に結合
    all_avail_actions = np.stack(avail_actions_df['array'].values)  # shape: (n_rows, n_agents, n_actions)
    all_pick_random = np.stack(pick_random_df['array'].values)      # shape: (n_rows, n_agents)
    all_picked_actions = np.stack(picked_actions_df['array'].values)  # shape: (n_rows, n_agents)

    # 「wait以外のアクションが選択可能で、pick_random が True」な状況のマスクを一度に作成
    not_only_wait = np.any(all_avail_actions[:, :, :-1], axis=2)  # shape: (n_rows, n_agents)
    mask = not_only_wait & all_pick_random  # shape: (n_rows, n_agents)

    # マスクに該当するpicked_actionsのみを抽出（1次元配列になる）
    filtered_actions = all_picked_actions[mask]

    # 集計
    num_actions = all_avail_actions.shape[2]
    unique_actions, counts = np.unique(filtered_actions, return_counts=True)

    # 結果を DataFrame に変換（全アクションを含む）
    total_counts = np.zeros(num_actions, dtype=int)
    total_counts[unique_actions] = counts

    result_df = pd.DataFrame({
        'action': range(num_actions),
        'count': total_counts
    })

    print("\n=== 集計結果 ===")
    print(result_df)
    print(f"\n総選択回数: {result_df['count'].sum()}")



=== run_id: pfn30lb0, name: 10a10f_lc1 (1/25) ===

=== 集計結果 ===
    action   count
0        0  424523
1        1  412723
2        2  425806
3        3  408723
4        4  432383
5        5  455358
6        6  417284
7        7  406729
8        8  434092
9        9  440166
10      10       0

総選択回数: 4257787

=== run_id: upnk5svz, name: 10a10f_lc1 (2/25) ===

=== 集計結果 ===
    action   count
0        0  414282
1        1  428241
2        2  489760
3        3  416016
4        4  442154
5        5  488022
6        6  391924
7        7  388167
8        8  410573
9        9  395862
10      10       0

総選択回数: 4265001

=== run_id: 60exp2wf, name: 10a10f_lc1 (3/25) ===

=== 集計結果 ===
    action   count
0        0  441582
1        1  425093
2        2  423788
3        3  434937
4        4  431710
5        5  420275
6        6  425799
7        7  423494
8        8  421933
9        9  415528
10      10       0

総選択回数: 4264139

=== run_id: 7qqhan3a, name: 10a10f_lc1 (4/25) ===


KeyboardInterrupt: 

# 

## fixed_wait_action_prob でpicked_actionsの分布がどう変わったか比較

In [37]:
run_from_fixed_wait_action_prob = None
run_from_conventional = None

print(runs)
runs = list(runs)
random.shuffle(runs)  # ランダムにシャッフルしてから選択
for run in runs:
    if not run_from_fixed_wait_action_prob and (run.config.get("enable_fixed_wait_action_prob") == True):
        run_from_fixed_wait_action_prob = run
    elif not run_from_conventional and (run.config.get("enable_fixed_wait_action_prob") == False):
        run_from_conventional = run

    if run_from_fixed_wait_action_prob and run_from_conventional:
        break
print("\n=== 選択された2つのrun ===")
print(f"run_from_fixed_wait_action_prob: id={run_from_fixed_wait_action_prob.id}, name={run_from_fixed_wait_action_prob.name}, enable_fixed_wait_action_prob={run_from_fixed_wait_action_prob.config.get('enable_fixed_wait_action_prob')}, fixed_wait_action_prob={run_from_fixed_wait_action_prob.config.get('fixed_wait_action_prob')}")
print(f"run_from_conventional: id={run_from_conventional.id}, name={run_from_conventional.name}, enable_fixed_wait_action_prob={run_from_conventional.config.get('enable_fixed_wait_action_prob')}")

# ここから分析
# まず各runについて、avail_actionsをロードして「not_only_wait & pick_random」の条件で絞り込む
def get_filtered_actions(run_id: str, name: str):
    avail_actions_df = load_array_log_df(run_id, "avail_actions")
    pick_random_df = load_array_log_df(run_id, "pick_random")
    picked_actions_df = load_array_log_df(run_id, "picked_actions")
    
    all_avail = np.stack(avail_actions_df['array'].values)
    all_pick_random = np.stack(pick_random_df['array'].values)
    all_picked = np.stack(picked_actions_df['array'].values)
    
    # not_only_wait: wait以外のアクションが選択可能
    not_only_wait = np.any(all_avail[:, :, :-1], axis=2)
    mask = not_only_wait & all_pick_random
    
    filtered = all_picked[mask]
    
    return all_picked, all_pick_random, filtered

all_picked_actions_fixed, all_pick_random_fixed, filtered_actions_fixed = get_filtered_actions(
    run_from_fixed_wait_action_prob.id, 
    run_from_fixed_wait_action_prob.name
)

all_picked_actions_conv, all_pick_random_conv, filtered_actions_conv = get_filtered_actions(
    run_from_conventional.id, 
    run_from_conventional.name
)

# not_only_wait & pick_random の条件で絞り込んだ集計
print("\n=== not_only_wait & all_pick_random での集計 ===")

print("\n--- run_from_fixed_wait_action_prob ---")
print(f"run_id: {run_from_fixed_wait_action_prob.id}, name: {run_from_fixed_wait_action_prob.name}")
unique_filtered_fixed, counts_filtered_fixed = np.unique(filtered_actions_fixed, return_counts=True)
num_actions = 11  # assuming 11 actions including wait
total_counts_filtered_fixed = np.zeros(num_actions, dtype=int)
total_counts_filtered_fixed[unique_filtered_fixed] = counts_filtered_fixed
result_df_filtered_fixed = pd.DataFrame({
    'action': range(num_actions),
    'count': total_counts_filtered_fixed
})
print(result_df_filtered_fixed)
print(f"総選択回数: {result_df_filtered_fixed['count'].sum()}")

print("\n--- run_from_conventional ---")
print(f"run_id: {run_from_conventional.id}, name: {run_from_conventional.name}")
unique_filtered_conv, counts_filtered_conv = np.unique(filtered_actions_conv, return_counts=True)
total_counts_filtered_conv = np.zeros(num_actions, dtype=int)
total_counts_filtered_conv[unique_filtered_conv] = counts_filtered_conv
result_df_filtered_conv = pd.DataFrame({
    'action': range(num_actions),
    'count': total_counts_filtered_conv
})
print(result_df_filtered_conv)
print(f"総選択回数: {result_df_filtered_conv['count'].sum()}")

print("\n=== run_from_fixed_wait_action_prob ===")
print(f"run_id: {run_from_fixed_wait_action_prob.id}, name: {run_from_fixed_wait_action_prob.name}")
picked_actions_df_fixed = load_array_log_df(run_from_fixed_wait_action_prob.id, "picked_actions")
pick_random_df_fixed = load_array_log_df(run_from_fixed_wait_action_prob.id, "pick_random")
all_picked_actions_fixed = np.stack(picked_actions_df_fixed['array'].values)
all_pick_random_fixed = np.stack(pick_random_df_fixed['array'].values)

# pick_random が True の時
picked_actions_random_fixed = all_picked_actions_fixed[all_pick_random_fixed]
unique_actions_random_fixed, counts_random_fixed = np.unique(picked_actions_random_fixed, return_counts=True)
result_df_random_fixed = pd.DataFrame({
    'action': unique_actions_random_fixed,
    'count': counts_random_fixed
})
print("\n[pick_random=True]")
print(result_df_random_fixed)
print(f"総選択回数: {result_df_random_fixed['count'].sum()}")

# pick_random が False の時
picked_actions_greedy_fixed = all_picked_actions_fixed[~all_pick_random_fixed]
unique_actions_greedy_fixed, counts_greedy_fixed = np.unique(picked_actions_greedy_fixed, return_counts=True)
result_df_greedy_fixed = pd.DataFrame({
    'action': unique_actions_greedy_fixed,
    'count': counts_greedy_fixed
})
print("\n[pick_random=False]")
print(result_df_greedy_fixed)
print(f"総選択回数: {result_df_greedy_fixed['count'].sum()}")

print("\n=== run_from_conventional ===")
print(f"run_id: {run_from_conventional.id}, name: {run_from_conventional.name}")
picked_actions_df_conv = load_array_log_df(run_from_conventional.id, "picked_actions")
pick_random_df_conv = load_array_log_df(run_from_conventional.id, "pick_random")
all_picked_actions_conv = np.stack(picked_actions_df_conv['array'].values)
all_pick_random_conv = np.stack(pick_random_df_conv['array'].values)

# pick_random が True の時
picked_actions_random_conv = all_picked_actions_conv[all_pick_random_conv]
unique_actions_random_conv, counts_random_conv = np.unique(picked_actions_random_conv, return_counts=True)
result_df_random_conv = pd.DataFrame({
    'action': unique_actions_random_conv,
    'count': counts_random_conv
})
print("\n[pick_random=True]")
print(result_df_random_conv)
print(f"総選択回数: {result_df_random_conv['count'].sum()}")

# pick_random が False の時
picked_actions_greedy_conv = all_picked_actions_conv[~all_pick_random_conv]
unique_actions_greedy_conv, counts_greedy_conv = np.unique(picked_actions_greedy_conv, return_counts=True)
result_df_greedy_conv = pd.DataFrame({
    'action': unique_actions_greedy_conv,
    'count': counts_greedy_conv
})
print("\n[pick_random=False]")
print(result_df_greedy_conv)
print(f"総選択回数: {result_df_greedy_conv['count'].sum()}")


[<Run kazuyasakakibara/FoodBank2025/d2lvfsb2 (finished)>, <Run kazuyasakakibara/FoodBank2025/z56flrfr (finished)>, <Run kazuyasakakibara/FoodBank2025/pfn30lb0 (finished)>, <Run kazuyasakakibara/FoodBank2025/mgdg9wbi (finished)>, <Run kazuyasakakibara/FoodBank2025/pm3prox0 (finished)>, <Run kazuyasakakibara/FoodBank2025/jejtnglb (finished)>, <Run kazuyasakakibara/FoodBank2025/f6rmqhl4 (finished)>, <Run kazuyasakakibara/FoodBank2025/anrcmwap (finished)>, <Run kazuyasakakibara/FoodBank2025/4xb6wr24 (finished)>, <Run kazuyasakakibara/FoodBank2025/z69ywi2r (finished)>, <Run kazuyasakakibara/FoodBank2025/p2oeb5ba (finished)>, <Run kazuyasakakibara/FoodBank2025/x8ga03bd (finished)>, <Run kazuyasakakibara/FoodBank2025/tl0ezist (finished)>, <Run kazuyasakakibara/FoodBank2025/yrg5e4rx (finished)>, <Run kazuyasakakibara/FoodBank2025/4bpf2x42 (finished)>, <Run kazuyasakakibara/FoodBank2025/hj2fyfik (finished)>, <Run kazuyasakakibara/FoodBank2025/yj4iq2ps (finished)>, <Run kazuyasakakibara/FoodBank